In [43]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.9MB/s 


In [0]:
link="https://drive.google.com/open?id=17FIPplAOkDN8i1tuT3nDTDegEfoAEyPL"

In [45]:
fluff, id = link.split('=')
print (id)

17FIPplAOkDN8i1tuT3nDTDegEfoAEyPL


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('news.csv')  
news = pd.read_csv('news.csv')

In [0]:
import pandas as pd

In [49]:
news.head()

,Unnamed: 0,article,category
0,24644,Global emissions targets will lead to 4C tempe...,CLIMATE
1,115540,Climate sceptics place bets on world cooling d...,SCIENCE
2,113058,This brain parasite can increase your chances ...,SOCIETY
3,79582,In pictures: RSPB's Big Garden Birdwatch - wha...,CLIMATE
4,76416,Duma wants convincing arguments for Kyoto rati...,BUSINESS


In [0]:
news=news.drop("Unnamed: 0",axis=1)

In [51]:
news.head()

,article,category
0,Global emissions targets will lead to 4C tempe...,CLIMATE
1,Climate sceptics place bets on world cooling d...,SCIENCE
2,This brain parasite can increase your chances ...,SOCIETY
3,In pictures: RSPB's Big Garden Birdwatch - wha...,CLIMATE
4,Duma wants convincing arguments for Kyoto rati...,BUSINESS


In [52]:
news["category"].unique()

array(['CLIMATE', 'SCIENCE', 'SOCIETY', 'BUSINESS', 'SCI-TECH', 'SPORTS',
       'ENTERTAINMENT', 'POLITICS', 'EDUCATION'], dtype=object)

In [53]:
pd.value_counts(news["category"])

SPORTS           22318
BUSINESS         21567
SCI-TECH         21240
CLIMATE          14770
SOCIETY          13873
SCIENCE          11293
EDUCATION         8292
ENTERTAINMENT     7364
POLITICS          6410
Name: category, dtype: int64

In [54]:
import nltk
import re
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()
nltk.download("stopwords")
stopwords_list=set(nltk.corpus.stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
news.isna().sum()

article     0
category    0
dtype: int64

In [56]:
def remove_stopwords(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token.lower() not in stopwords_list]
  filtered_tokens=" ".join(filtered_tokens)
  return filtered_tokens
remove_stopwords("This is an apple")

'apple'

In [57]:
def remove_special_char(text):
  pattern=r'[^a-zA-Z\s]'
  text=re.sub(pattern,'',text)
  return text
remove_special_char("Hey @ asdhasd123")

'Hey  asdhasd'

In [58]:
#def lemma(text):
#  text=[lemmatizer.lemmatize(word=text,pos="v") for word in text]
#  return text

print(lemmatizer.lemmatize("are",pos='v'))

be


In [80]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
def normalize_corpus(corpus):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # lowercase the text    
        doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        doc =stemmer(doc)
        # remove special characters and\or digits    
        special_char_pattern = re.compile(r'([{.(-)!}])')
        doc = special_char_pattern.sub(" \\1 ", doc)
        doc = remove_special_char(doc)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        doc = remove_stopwords(doc)
        normalized_corpus.append(doc)
        
    return normalized_corpus

In [89]:
news["Processed_article"]=normalize_corpus(news["article"])
norm_corpus=list(news["Processed_article"])
#showing sample
news.iloc[1][["Processed_article","article"]].to_dict()

{'Processed_article': 'climat sceptic place bet world cool solar physicist make wager climat modeller british climat model ha final persuad globalwarm sceptic wager money contrarian predict climat change jam annan base japan agenc marineearth scienc technolog yokohama ha agre us bet galina mashnich vladimir bashkirtsev two solar physicist argu global temperatur driven chang sun activ fall next decade bet side say formal legal document came climat sceptic refus wager money annan began hi quest last winter hear richard lindzen meteorologist massachusett institut technolog question extent human activ influenc climate say wa bet global temperatur drop next years payoff retir age would nice topup pension say annan wager wa ever agreed annan say lindzen want odd fall temperatures thi meant annan would pay temperatur dropped receiv onli rose total annan say tri fail agre term seven sceptics two russian climat sceptic ride predict reduct sunspot activ lower earth temperature image millerdmioth

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re


In [87]:
def stemmer(text):
        text=' '.join([ps.stem(word) for word in text.split()])
        return text
stemmer("cooling")

'cool'

In [106]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(news['Processed_article'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 360417 unique tokens.


In [107]:
X = tokenizer.texts_to_sequences(news['Processed_article'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (127127, 250)


In [108]:
Y = pd.get_dummies(news['category']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (127127, 9)


In [109]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 2019)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(114414, 250) (114414, 9)
(12713, 250) (12713, 9)


In [110]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(9, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 250, 100)          10000000  
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 250, 100)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 909       
Total params: 10,081,309
Trainable params: 10,081,309
Non-trainable params: 0
_________________________________________________________________
None


In [111]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train on 102972 samples, validate on 11442 samples
Epoch 1/5
102972/102972 [==============================] - 1343s 13ms/step - loss: 0.4284 - acc: 0.8484 - val_loss: 0.2045 - val_acc: 0.9358
Epoch 2/5
102972/102972 [==============================] - 1342s 13ms/step - loss: 0.1584 - acc: 0.9496 - val_loss: 0.2113 - val_acc: 0.9339
Epoch 3/5
102972/102972 [==============================] - 1345s 13ms/step - loss: 0.1076 - acc: 0.9659 - val_loss: 0.1911 - val_acc: 0.9414
Epoch 4/5
102972/102972 [==============================] - 1858s 18ms/step - loss: 0.0730 - acc: 0.9763 - val_loss: 0.2352 - val_acc: 0.9345
Epoch 5/5
102972/102972 [==============================] - 1540s 15ms/step - loss: 0.0517 - acc: 0.9831 - val_loss: 0.2127 - val_acc: 0.9432


In [112]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

12713/12713 [==============================] - 26s 2ms/step
Test set
  Loss: 0.198
  Accuracy: 0.944


In [126]:
news['category'].value_counts()

SPORTS           22318
BUSINESS         21567
SCI-TECH         21240
CLIMATE          14770
SOCIETY          13873
SCIENCE          11293
EDUCATION         8292
ENTERTAINMENT     7364
POLITICS          6410
Name: category, dtype: int64

In [122]:
new_complaint = ["Climate sceptics place bets on world cooling down. Solar physicists make $10,000 wager with climate modeller.A British climate modeller has finally persuaded global-warming sceptics to wager money on their contrarian predictions about climate change.James Annan, who is based at the Japan Agency for Marine-Earth Science and Technology in Yokohama, has agreed a US$10,000 bet with Galina Mashnich and Vladimir Bashkirtsev, two solar physicists who argue that global temperatures are driven by changes in the Sun's activity and will fall over the next decade. The bet, which both sides say they are willing to formalize in a legal document, came after other climate sceptics refused to wager money.Annan began his quest last winter after hearing Richard Lindzen, a meteorologist at the Massachusetts Institute of Technology who questions the extent to which human activities are influencing climate, say he was willing to bet that global temperatures will drop over the next 20 years. “A pay-off at retirement age would be a nice top-up to my pension,” says Annan.But no wager was ever agreed. Annan says that Lindzen wanted odds of 50-to-1 against falling temperatures: this meant that Annan would pay out $10,000 if temperatures dropped, but receive only $200 if they rose. In total, Annan says he tried and failed to agree terms with seven sceptics.Two Russian climate sceptics have $10,000 riding on their prediction that a reduction in sunspot activity will lower Earth's temperature. Image: D. MILLER/DMIOther potential climate gamblers have drawn a blank with their attempts to enter similar bets with climate-change sceptics. In May, environmental activist George Monbiot challenged climate sceptic Myron Ebell to a £5,000 (US$9,000) wager live on BBC radio. Ebell, a global-warming specialist at the Competitive Enterprise Institute, a think-tank in Washington DC, declined, saying he has four children to put through university and so does not “want to take risks”.But Annan's search ended with Mashnich and Bashkirtsev, who are based at the Institute of Solar-Terrestrial Physics in Irkutsk, Russia. They say that global surface air temperatures closely correlate with the size and number of sunspots. Sunspot levels follow regular patterns and the Sun is expected to be in a less active phase over the next few decades, leading Mashnich and Bashkirtsev to predict a drop in temperature.Both sides have agreed to compare the average global surface temperature between 1998 and 2003 with that between 2012 and 2017, as defined by the records of the US National Climatic Data Center. If the temperature drops, Annan will pay Mashnich and Bashkirtsev $10,000 in 2018, with the same sum going the other way if the temperature rises.Piers Corbyn, head of Weather Action, a private meteorological service based in London, told Nature he would like to enter into a similar bet. Corbyn's theory, the details of which he has not revealed, predicts that changes in solar activity will cause “considerable world cooling” by 2040. Annan challenged him to a bet in May, but Corbyn says he did not receive the e-mail. “I'm happy to bet loads of money,” he says.Rights and permissions To obtain permission to re-use content from this article visit RightsLink."]
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels=["SPORTS","BUSINESS","SCI-TECH","CLIMATE","SOCIETY","EDUCATION","SCIENCE","ENTERTAINMENT","POLITICS"]
print(pred, labels[np.argmax(pred)])

[[1.3338857e-05 3.0617188e-03 2.0277448e-05 2.2006576e-05 1.9315248e-05
  2.4349978e-05 9.9671376e-01 1.1458261e-04 1.0516973e-05]] SCIENCE


In [0]:
import numpy as np